In [ ]:
# ============================================================
# [Re] Watts–Strogatz Small-World Networks
# - CONSISTENT METRICS on giant component (Gc)
# - GC FILTER (φ>=0.95) for apples-to-apples normalized C/L curves
# - FULL-GRAPH ROBUSTNESS: global efficiency E(G) across all p
# - THEORY + EMPIRICAL random baselines for C and L
# - REWIRING VALIDATION: lattice-edge fraction vs p
# - DEGREE + ASSORTATIVITY diagnostics vs p
# - 50 seeds + CI stability analysis (L at p=1e-2)
# - Outputs: CSVs, PNGs, metadata.json, and a printed validation report
# ============================================================

!pip install -q networkx==3.3

import os, sys, json, time, math, csv, random
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

# -------- Determinism --------
os.environ["PYTHONHASHSEED"] = "0"
random.seed(0)
np.random.seed(0)

# -------- Paths --------
ROOT = Path("results"); FIGS = ROOT/"figs"; DATA = ROOT/"data"
for d in (FIGS, DATA): d.mkdir(parents=True, exist_ok=True)

# -------- Experiment params --------
N = 1000              # nodes
k = 10                # mean degree (must be even)
assert k % 2 == 0
# p-grid: 0 plus logspaced 1e-4..1
p_values = np.unique(np.concatenate([[0.0], np.round(np.logspace(-4, 0, 13), 4)]))
n_seeds = 50          # statistical power for 95% CIs
MIN_GC_FRAC = 0.95    # apples-to-apples threshold for normalized curves

# -------- Helpers --------
def mean_ci95(x):
    x = np.asarray(x, float)
    m = x.mean()
    if len(x) < 2:
        return float(m), 0.0
    s = x.std(ddof=1) / max(len(x)**0.5, 1e-12)
    return float(m), float(1.96 * s)

def ws_graph(N, k, p, seed):
    return nx.watts_strogatz_graph(N, k, float(p), seed=seed)

def to_giant_component_view(G):
    if nx.is_connected(G):
        return G, 1.0
    nodes = max(nx.connected_components(G), key=len)
    Gc = G.subgraph(nodes)  # view, no copy
    return Gc, Gc.number_of_nodes() / G.number_of_nodes()

def graph_metrics_consistent(G):
    Gc, frac = to_giant_component_view(G)
    C = nx.average_clustering(Gc)
    L = nx.average_shortest_path_length(Gc)
    return C, L, frac, Gc

def degree_stats(G):
    deg = np.array([d for _, d in G.degree()], dtype=float)
    return float(deg.mean()), float(deg.var())

def assortativity(G):
    try:
        return float(nx.degree_assortativity_coefficient(G))
    except Exception:
        return float('nan')

def ring_edges_set(N, k):
    half = k // 2
    S = set()
    for u in range(N):
        for d in range(1, half+1):
            v = (u + d) % N
            a, b = sorted((u, v))
            S.add((a, b))
    return S

RING = ring_edges_set(N, k)

def fraction_lattice_edges(G):
    Eset = set(tuple(sorted(e)) for e in G.edges())
    common = len(Eset & RING)
    return common / max(len(Eset), 1)

def proper_random_theory_baselines(N, k):
    C_rand_th = k / (N - 1) if N > 1 else float('nan')
    L_rand_th = math.log(N) / math.log(k) if (N > 1 and k > 1) else float('nan')
    return C_rand_th, L_rand_th

def empirical_ws_random_baselines(N, k, n_samples=30):
    C_vals, L_vals, E_vals = [], [], []
    for seed in range(n_samples):
        G = nx.watts_strogatz_graph(N, k, 1.0, seed=seed)
        Gc, _ = to_giant_component_view(G)
        C_vals.append(nx.average_clustering(Gc))
        L_vals.append(nx.average_shortest_path_length(Gc))
        E_vals.append(nx.global_efficiency(G))
    return float(np.mean(C_vals)), float(np.mean(L_vals)), float(np.mean(E_vals))

def ci_stability(p_target=1e-2, max_seeds=50, step=5):
    means, half, ns = [], [], []
    for m in range(step, max_seeds+1, step):
        Ls = []
        for s in range(m):
            G = ws_graph(N, k, p_target, seed=s)
            C, L, frac, _ = graph_metrics_consistent(G)
            Ls.append(L)
        mmean, mci = mean_ci95(Ls)
        means.append(mmean); half.append(mci); ns.append(m)
    return ns, means, half

# -------- Baselines --------
C_rand_th, L_rand_th = proper_random_theory_baselines(N, k)
C_rand_emp, L_rand_emp, E_rand_emp = empirical_ws_random_baselines(N, k, n_samples=30)

# -------- Baseline p=0 (processed identically on Gc) --------
C0_vals, L0_vals, phi0_vals = [], [], []
for s in range(n_seeds):
    G0 = ws_graph(N, k, 0.0, seed=s)
    C0, L0, phi0, _ = graph_metrics_consistent(G0)
    C0_vals.append(C0); L0_vals.append(L0); phi0_vals.append(phi0)
C0_mean, C0_ci = mean_ci95(C0_vals)
L0_mean, L0_ci = mean_ci95(L0_vals)
phi0_mean, phi0_ci = mean_ci95(phi0_vals)

# -------- Sweep over p --------
rows = []
for p in p_values:
    C_vals, L_vals, phi_vals = [], [], []
    E_full_vals, deg_means, deg_vars, assort_vals, latfrac_vals = [], [], [], [], []
    disc_count = 0
    for s in range(n_seeds):
        G = ws_graph(N, k, p, seed=s)
        if not nx.is_connected(G):
            disc_count += 1
        # metrics on Gc
        C, L, phi, Gc = graph_metrics_consistent(G)
        C_vals.append(C); L_vals.append(L); phi_vals.append(phi)
        # full-graph robustness
        E_full_vals.append(nx.global_efficiency(G))
        # diagnostics
        mu, var = degree_stats(G)
        deg_means.append(mu); deg_vars.append(var)
        assort_vals.append(assortativity(G))
        latfrac_vals.append(fraction_lattice_edges(G))
    Cm, Ce = mean_ci95(C_vals)
    Lm, Le = mean_ci95(L_vals)
    phim, phie = mean_ci95(phi_vals)
    Em, Ee = mean_ci95(E_full_vals)
    dmu, dve = mean_ci95(deg_means)
    dvarm, dvare = mean_ci95(deg_vars)
    r_m, r_e = mean_ci95(assort_vals)
    latm, late = mean_ci95(latfrac_vals)
    disc_frac = disc_count / n_seeds

    rows.append({
        "p": float(p),
        # absolute metrics on Gc
        "C_mean": Cm, "C_ci95": Ce,
        "L_mean": Lm, "L_ci95": Le,
        # normalized to p=0 (Gc)
        "C_over_C0": Cm / max(C0_mean, 1e-12),
        "L_over_L0": Lm / max(L0_mean, 1e-12),
        # theory baselines (ER-like)
        "C_over_Crand_th": Cm / max(C_rand_th, 1e-12),
        "L_over_Lrand_th": Lm / max(L_rand_th, 1e-12),
        # empirical WS p=1 baselines
        "C_over_Crand_emp": Cm / max(C_rand_emp, 1e-12),
        "L_over_Lrand_emp": Lm / max(L_rand_emp, 1e-12),
        # connectivity + robustness
        "GC_frac_mean": phim, "GC_frac_ci95": phie,
        "disconnected_fraction": disc_frac,
        "E_full_mean": Em, "E_full_ci95": Ee,
        # diagnostics
        "deg_mean": dmu, "deg_mean_ci95": dve,
        "deg_var": dvarm, "deg_var_ci95": dvare,
        "assort_mean": r_m, "assort_ci95": r_e,
        "lattice_edge_frac_mean": latm, "lattice_edge_frac_ci95": late,
        # constants
        "N": N, "k": k, "seeds": n_seeds
    })

# -------- Save metrics CSV --------
metrics_csv = DATA / "ws_metrics_pubready.csv"
with open(metrics_csv, "w", newline="") as f:
    w = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
    w.writeheader(); w.writerows(rows)

# -------- Load for plotting --------
data = {r["p"]: r for r in rows}
ps = np.array(sorted(data.keys()))

def arr(key):
    return np.array([data[p][key] for p in ps])

C_rel = arr("C_over_C0"); L_rel = arr("L_over_L0")
GC_frac = arr("GC_frac_mean")
valid_mask = GC_frac >= MIN_GC_FRAC

# ---- Fig 1: Normalized C/C0 and L/L0 (Gc) with φ filter ----
plt.figure(figsize=(9,5))
plt.plot(ps[valid_mask], C_rel[valid_mask], marker="o", label="C/C(0) (Gc, φ≥0.95)")
plt.plot(ps[valid_mask], L_rel[valid_mask], marker="s", label="L/L(0) (Gc, φ≥0.95)")
# faint outside valid region
if (~valid_mask).any():
    plt.plot(ps[~valid_mask], C_rel[~valid_mask], alpha=0.3)
    plt.plot(ps[~valid_mask], L_rel[~valid_mask], alpha=0.3)
plt.xscale("symlog", linthresh=1e-4)
plt.xlabel("p"); plt.ylabel("normalized")
plt.title(f"WS normalized on giant component (N={N}, k={k}, seeds={n_seeds})")
plt.grid(True, linewidth=0.4); plt.legend()
plt.tight_layout(); plt.savefig(FIGS/"ws_CL_normalized_gc_filtered.png", dpi=240); plt.close()

# ---- Fig 2: Full-graph global efficiency across all p ----
E_mean = arr("E_full_mean")
plt.figure(figsize=(9,5))
plt.plot(ps, E_mean, marker="o", label="Global efficiency E(G) (full graph)")
plt.xscale("symlog", linthresh=1e-4)
plt.xlabel("p"); plt.ylabel("E(G)")
plt.title("Full-graph efficiency across p (robustness over disconnections)")
plt.grid(True, linewidth=0.4); plt.legend()
plt.tight_layout(); plt.savefig(FIGS/"ws_efficiency_fullgraph.png", dpi=240); plt.close()

# ---- Fig 3: Lattice-edge fraction vs p (rewiring validation) ----
latfrac = arr("lattice_edge_frac_mean")
plt.figure(figsize=(9,5))
plt.plot(ps, latfrac, marker="o", label="Fraction of lattice edges remaining")
plt.xscale("symlog", linthresh=1e-4)
plt.xlabel("p"); plt.ylabel("fraction")
plt.title("Rewiring validation: lattice-edge fraction vs p")
plt.grid(True, linewidth=0.4); plt.legend()
plt.tight_layout(); plt.savefig(FIGS/"ws_lattice_fraction.png", dpi=240); plt.close()

# ---- Fig 4: CI stability for L at p = 1e-2 ----
ns, means, half = ci_stability(p_target=1e-2, max_seeds=n_seeds, step=5)
plt.figure(figsize=(7,4))
plt.plot(ns, half, marker="o")
plt.xlabel("# seeds"); plt.ylabel("95% CI half-width (L at p=1e-2)")
plt.title("CI stability diagnostic")
plt.grid(True, linewidth=0.4); plt.tight_layout()
plt.savefig(FIGS/"ws_ci_stability_L_p1e-2.png", dpi=220); plt.close()

# ---- Optional: Baseline comparison plots to theory/empirical ----
C_over_Crand_th = arr("C_over_Crand_th")
L_over_Lrand_th = arr("L_over_Lrand_th")
C_over_Crand_emp = arr("C_over_Crand_emp")
L_over_Lrand_emp = arr("L_over_Lrand_emp")

plt.figure(figsize=(9,5))
plt.plot(ps, C_over_Crand_th, marker="o", label="C / C_rand_theory")
plt.plot(ps, L_over_Lrand_th, marker="s", label="L / L_rand_theory")
plt.xscale("symlog", linthresh=1e-4)
plt.xlabel("p"); plt.ylabel("ratio to theory baseline")
plt.title("Theory baseline normalization (ER-like approximations)")
plt.grid(True, linewidth=0.4); plt.legend()
plt.tight_layout(); plt.savefig(FIGS/"ws_theory_baseline_ratios.png", dpi=240); plt.close()

plt.figure(figsize=(9,5))
plt.plot(ps, C_over_Crand_emp, marker="o", label="C / C_rand_emp (WS p=1)")
plt.plot(ps, L_over_Lrand_emp, marker="s", label="L / L_rand_emp (WS p=1)")
plt.xscale("symlog", linthresh=1e-4)
plt.xlabel("p"); plt.ylabel("ratio to empirical WS p=1")
plt.title("Empirical WS(p=1) baseline normalization")
plt.grid(True, linewidth=0.4); plt.legend()
plt.tight_layout(); plt.savefig(FIGS/"ws_empirical_baseline_ratios.png", dpi=240); plt.close()

# -------- Metadata --------
meta = {
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "python": sys.version,
    "numpy": np.__version__,
    "matplotlib": plt.matplotlib.__version__,
    "networkx": nx.__version__,
    "params": {
        "N": N, "k": k, "seeds": n_seeds,
        "p_values": p_values.tolist(),
        "MIN_GC_FRAC": MIN_GC_FRAC
    },
    "baselines": {
        "C_rand_theory": C_rand_th,
        "L_rand_theory": L_rand_th,
        "C_rand_empirical_WS_p1": C_rand_emp,
        "L_rand_empirical_WS_p1": L_rand_emp,
        "E_rand_empirical_WS_p1": E_rand_emp
    }
}
with open(ROOT/"metadata.json","w") as f:
    json.dump(meta, f, indent=2)

# -------- Validation report (printed) --------
# Small-world window heuristic on valid region
valid_ps = ps[valid_mask]
valid_Crel = C_rel[valid_mask]
valid_Lrel = L_rel[valid_mask]
sw_mask = (valid_Crel > 0.5) & (valid_Lrel < 0.2)
sw_window = (float(valid_ps[sw_mask].min()), float(valid_ps[sw_mask].max())) if sw_mask.any() else None

print("=== RUN COMPLETE ===")
print("Figures saved to:", str(FIGS))
print("Metrics CSV saved to:", str(metrics_csv))
print("\n--- Validation Report ---")
print(f"p=0 (ring) -> C0_mean={C0_mean:.3f} (high expected), L0_mean={L0_mean:.3f} (large expected)")
print(f"Theory baselines: C_rand_th={C_rand_th:.4f}, L_rand_th={L_rand_th:.3f}")
print(f"Empirical WS(p=1) baselines: C_rand_emp={C_rand_emp:.4f}, L_rand_emp={L_rand_emp:.3f}, E_rand_emp={E_rand_emp:.4f}")
print(f"GC φ at p=0: mean={phi0_mean:.3f} ± {phi0_ci:.3f}")
print(f"Small-world window (heuristic, φ≥{MIN_GC_FRAC}): {sw_window}")
print("CI stability figure written to figs/ws_ci_stability_L_p1e-2.png")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.3 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/networkx/algorithms/assortativity/correlation.py:302: RuntimeWarning: invalid value encountered in scalar divide
  return float((xy * (M - ab)).sum() / np.sqrt(vara * varb))


=== RUN COMPLETE ===
Figures saved to: results/figs
Metrics CSV saved to: results/data/ws_metrics_pubready.csv

--- Validation Report ---
p=0 (ring) -> C0_mean=0.667 (high expected), L0_mean=50.450 (large expected)
Theory baselines: C_rand_th=0.0100, L_rand_th=3.000
Empirical WS(p=1) baselines: C_rand_emp=0.0089, L_rand_emp=3.268, E_rand_emp=0.3235
GC φ at p=0: mean=1.000 ± 0.000
Small-world window (heuristic, φ≥0.95): (0.01, 0.1)
CI stability figure written to figs/ws_ci_stability_L_p1e-2.png


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls


sample_data
